In [ ]:
from google.colab import auth
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, roc_auc_score, roc_curve, 
                             confusion_matrix, f1_score, precision_score, 
                             recall_score)

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer


## Loading Data


In [ ]:
auth.authenticate_user()

#### Lab Data

In [ ]:
!gsutil cp gs://mlhc-mimic/adult_icu.gz ./


In [ ]:
lab_df = pd.read_csv('adult_icu.gz')

#### Note Data

In [ ]:
!gsutil cp gs://mlhc-mimic/adult_notes.gz ./


In [ ]:
note_df = pd.read_csv('adult_notes.gz')

## Question 2

Predicting hospital mortality from lab values



In [ ]:
##Explore the dataset lab_df

In [ ]:
len(lab_df)

In [ ]:
lab_df["hadm_id"].nunique()

In [ ]:
lab_df["subject_id"].nunique()

In [ ]:
##Dropping features
lab_df.drop(["subject_id","hadm_id","icustay_id", "mort_icu", "mort_oneyr", "adult_icu", "admType_NEWBORN"], axis=1, inplace=True)

In [ ]:
##Split the dataset into train/val/test. Note that we have already provided
##the columns "train", "test", "valid" for you which splits the dataset into 
##training set, validation set and testing set. 
##Once you're done, remove the columns train, val and test from the dataset. 

In [ ]:
training_data = lab_df[lab_df["train"] == 1].drop(columns=["train", "test", "valid"])
validation_data = lab_df[lab_df["valid"] == 1].drop(columns=["train", "test", "valid"])
test_data = lab_df[lab_df["test"] == 1].drop(columns=["train", "test", "valid"])

In [ ]:
print(len(training_data))
print(len(validation_data))
print(len(test_data))

In [ ]:
train_X, train_y = training_data.drop(columns=["mort_hosp"]), training_data["mort_hosp"]
valid_X, valid_y = validation_data.drop(columns=["mort_hosp"]), validation_data["mort_hosp"]
test_X, test_y = test_data.drop(columns=["mort_hosp"]), test_data["mort_hosp"]

In [ ]:
sum(train_y.values) / len(train_y)

In [ ]:
##Normalize the data in train/val/test. Be sure to fit StandardScaler to the training dataset only! 

In [ ]:
pipeline = Pipeline(
    [("num_standardizer", StandardScaler()),
     ("clf", LogisticRegression(solver='liblinear', max_iter=2000, verbose=2))
     ]
)

In [ ]:
##Problem 2.5, 2.6 - Train a Logistic Regression model (with solver = 'libnear') to predict mortality given the remaining features available. 

C = [0.1, 0.25, 1]
penalty = ['l1', 'l2']

In [ ]:
for c_val in C:
  for penalty_val in penalty:
    pipeline.set_params(clf__C = c_val, clf__penalty = penalty_val)
    pipeline.fit(train_X, train_y)
    print(f"c = {c_val}, penalty = {penalty_val}")
    print(pipeline.score(valid_X, valid_y))

In [ ]:
pipeline.set_params(clf__C = 1, clf__penalty = 'l2')
pipeline.fit(train_X, train_y)

In [ ]:
pipeline.score(test_X, test_y)

In [ ]:
standardizer = pipeline['num_standardizer']
model = pipeline['clf']

In [ ]:
test_X_standardized = standardizer.transform(test_X)
predicted_values = model.predict(test_X_standardized)

In [ ]:
accuracy_score(test_y, predicted_values)

In [ ]:
roc_auc_score(test_y, predicted_values)

In [ ]:
confusion_matrix(test_y, predicted_values)

In [ ]:
print(precision_score(test_y, predicted_values))
print(recall_score(test_y, predicted_values))

In [ ]:
##Problem 2.7 - Which of the following features are among the top 5 most 
##positive features, based on the coefficients of the logistic regression model?

In [ ]:
feature_importances = zip(train_X.columns, *model.coef_)
print(sorted(list(feature_importances), key=lambda x: -x[1]))

In [ ]:
##Problem 2.8 - Which of the following features are among the top 5 most 
##negative features, based on the coefficients of the logistic regression model?

In [ ]:
feature_importances = zip(train_X.columns, *model.coef_)
print(sorted(list(feature_importances), key=lambda x: x[1]))

## Question 3

Predicting hospital mortality from clinical notes


In [ ]:
len(note_df)

In [ ]:
note_df.drop(["subject_id","hadm_id","icustay_id", "mort_icu", "mort_oneyr"], axis=1, inplace=True)

In [ ]:
note_df.columns

In [ ]:
##Split the dataset into train/val/test

In [ ]:
training_data = note_df[note_df["train"] == 1].drop(columns=["train", "test", "valid"])
validation_data = note_df[note_df["valid"] == 1].drop(columns=["train", "test", "valid"])
test_data = note_df[note_df["test"] == 1].drop(columns=["train", "test", "valid"])

In [ ]:
print(len(training_data))
print(len(validation_data))
print(len(test_data))

In [ ]:
train_X, train_y = training_data["chartext"].values, training_data["mort_hosp"].values
valid_X, valid_y = validation_data["chartext"].values, validation_data["mort_hosp"].values
test_X, test_y = test_data["chartext"].values, test_data["mort_hosp"].values

In [ ]:
##Fit a CountVectorizer with max_features = 5000 to the trianing dataset and generate features for train/val/test. 
vectorizer = CountVectorizer(max_features = 5000)

In [ ]:
pipeline = Pipeline([
                     ("vectorizer", vectorizer),
                     ("clf", LogisticRegression(solver='liblinear', max_iter=2000))
])

In [ ]:
##Problem 3.1, 3.2 Train a Logistic Regression model (with solver = 'liblinear') to predict mortality given the remaining features available. 

C = [0.1,0.25,1]
penalty = ['l1','l2']
for c_val in C:
  for penalty_val in penalty:
    print(f"using C = {c_val}, penalty = {penalty_val}")
    pipeline.set_params(clf__C = c_val, clf__penalty = penalty_val)
    pipeline.fit(train_X, train_y)
    print(f"validation accuracy = {pipeline.score(valid_X, valid_y)}")

In [ ]:
## retraining the model on best hyper parameters found using validation dataset
pipeline.set_params(clf__C = 0.1, clf__penalty = 'l1')
pipeline.fit(train_X, train_y)

In [ ]:
featurizer, model = pipeline["vectorizer"], pipeline["clf"]
test_X_featurized = featurizer.transform(test_X)
predicted_values = model.predict(test_X_featurized)

print(f"accuracy on test data = {accuracy_score(test_y, predicted_values)}")
print(f"auc on test data = {roc_auc_score(test_y, predicted_values)}")
print(f"f1score on test data = {f1_score(test_y, predicted_values)}")

In [ ]:
vocab_idxs = featurizer.vocabulary_

In [ ]:
vocab_used = [None]*len(vocab_idxs)
for token, idx in vocab_idxs.items():
  vocab_used[idx] = token

In [ ]:
##Problem 3.3 Which of the following features are among the top 5 most 
##predictive positive words, based on the coefficients of the logistic regression model?
feature_importances = zip(vocab_used, *model.coef_)
pos_predictive_tokens = sorted(list(feature_importances), key=lambda x: -x[1])
print(pos_predictive_tokens[:5])

In [ ]:
##Problem 3.4 Which of the following features are among the top 5 most 
##predictive negative words, based on the coefficients of the logistic regression model?
feature_importances = zip(vocab_used, *model.coef_)
neg_predictive_tokens = sorted(list(feature_importances), key=lambda x: x[1])
print(neg_predictive_tokens[:5])

## Question 4

Analysis of data and results

In [ ]:
##Problem 4.1 - people / mortality rate in different ethnic categorizations

In [ ]:
len(lab_df)

In [ ]:
lab_df.columns

In [ ]:
for col in ['eth_asian', 'eth_black', 'eth_hispanic', 'eth_white', 'eth_other']:
  eth_entries = lab_df[lab_df[col] == 1]
  num_entries = len(eth_entries)
  print(f"Number of people with ethnicity {col[4:]} = {num_entries}")
  num_mortalities = len(eth_entries[eth_entries["mort_hosp"] == 1])
  print(f"Hospital mortality rate = {num_mortalities / num_entries}")

In [ ]:
##Problem 4.2 - plot histogram for ages

In [ ]:
lab_df['age'].describe()

In [ ]:
ages_20_to_90 = lab_df[(lab_df['age'] >= 20) & (lab_df['age'] <= 90)]
len(ages_20_to_90)

In [ ]:
plt.hist(ages_20_to_90["age"].values)

# New Section

In [ ]:
##TODO: Problem 4.3 - plot histogram for mortality rates

In [ ]:
lab_df["age_floor"] = lab_df["age"] // 10

In [ ]:
plot_data = lab_df.groupby(["age_floor"]).mean()["mort_hosp"]
plot_data

In [ ]:
plt.bar(x = plot_data.index, height = plot_data)

In [ ]:
##Problem 4.4 - retrain a model using C=1, penalty = l2 and evaluate AUC
##and accuracy on the test set with age less than 40 and on the test set with
##age greater than or equal to 40.

In [ ]:
lab_df.drop(columns = ["age_floor", "mortality_rate"], inplace=True)

In [ ]:
training_data = lab_df[lab_df["train"] == 1].drop(columns=["train", "test", "valid"])
validation_data = lab_df[lab_df["valid"] == 1].drop(columns=["train", "test", "valid"])
test_data = lab_df[lab_df["test"] == 1].drop(columns=["train", "test", "valid"])

In [ ]:
print(len(training_data))
print(len(validation_data))
print(len(test_data))

In [ ]:
train_X, train_y = training_data.drop(columns=["mort_hosp"]), training_data["mort_hosp"]
valid_X, valid_y = validation_data.drop(columns=["mort_hosp"]), validation_data["mort_hosp"]

In [ ]:
len(train_X.columns)

### Retrain the model

In [ ]:
pipeline = Pipeline(
    [("num_standardizer", StandardScaler()),
     ("clf", LogisticRegression(solver='liblinear', 
                                C = 1, penalty = 'l2',
                                max_iter=2000, verbose=2))
     ]
)

In [ ]:
pipeline.fit(train_X, train_y)

In [ ]:
standardizer, model = pipeline["num_standardizer"], pipeline["clf"]

In [ ]:
patients_less_than_40 = test_data[test_data["age"] < 40]
patients_more_than_40 = test_data[test_data["age"] >= 40]

In [ ]:
print(f"Number of patients admitted below the age of 40 = {len(patients_less_than_40)}")
print(f"Number of patients admitted equal to and above 40 = {len(patients_more_than_40)}")

In [ ]:
for data_df in [patients_less_than_40, patients_more_than_40]:
  features, labels = data_df.drop(columns=["mort_hosp"]), data_df["mort_hosp"]
  features_standardized = standardizer.transform(features)
  predictions = model.predict(features_standardized)
  print(f"min age = {data_df['age'].min()}, max age = {data_df['age'].max()}")
  print(f"model accuracy = {accuracy_score(labels, predictions)}")
  print(f"auc score = {roc_auc_score(labels, predictions)}")
  print()